# Getting Started : Basic Inputs and Outputs

If you are here then you have already successfully 

1) Installed the code

2) Downloaded the necessary reference data

3) Added ``export picaso_refdata="/path/to/picaso/reference"`` to ~/.bash_profile

If you have not done these things, please return to [Installation Guilde](https://natashabatalha.github.io/picaso/installation.html)

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd

#picaso
from picaso import justdoit as jdi 
from picaso import justplotit as jpi
#plotting
from bokeh.io import output_notebook
output_notebook()
from bokeh.plotting import show

Loading BokehJS ...

First, let's load in the inputs needed for PICASO. This will be a dict formatted input structure. Let's spend some time learning this.

In [2]:
all_inputs = jdi.load_inputs()

In [3]:
all_inputs.keys()

dict_keys(['planet', 'star', 'atmosphere', 'approx', 'disco', 'opacities', 'test_mode'])

In order to run the code we need (at the minimum) specific info about the:

- **planet** : gravity

- **star** : temperature, metallicity, gravity

- **atmosphere** : P-T profile, chemical composition, cloud properties (discussed later)

Additionally, we have some optional keys that will be discussed in later notebooks: 

- **approx** : approximations to rapidly compute reflected light 

- **disco** : number of individual facets you want to compute before integrating to 1D spectrum (think disco ball) 

- **opacities** : keeping track of opacity files used 

- **test_mode** : used to do benchmark testing 

## Set Planet & Star Properties 

In [4]:
#gravity 
all_inputs['planet']['gravity'] = 25
all_inputs['planet']['gravity_unit'] = 'm/(s**2)' #any astropy units are okay here 
#can also enter in radius and mass 
#all_inputs['planet']['mass']=1
#all_inputs['planet']['mass_unit'] = 'M_earth' (radius would be the same but with 'radius' and 'radius_unit')

#add star properties 
all_inputs['star']['temp'] = 6000 #kelvin
all_inputs['star']['metal'] = 0.0122 #log metal
all_inputs['star']['logg'] = 4.437 #log cgs

## Set Atmospheric Composition

There are three different options for setting atmospheric composition. 

    1) Specifying a file path (instructions on formatting are below)
    2) Give arbitrary pressure, temperature and composition directly as a dictionary input 
    3) Specify a file path, but only run a subset of the molecules included in the file (mostly used for testing)

### Option 1) Specify file path 

Below, I am loading in a profile path for Jupiter that should be included in your reference data

In [5]:
print(jdi.jupiter_pt()) #should return the path to your reference data

/Users/natashabatalha/Documents/PICASO/picaso/reference/base_cases/jupiter.pt


In [6]:
all_inputs['atmosphere']['profile']['filepath'] = jdi.jupiter_pt() 

### File format Specifications
1) Must have header which includes.. 
- At least **pressure**(bars), **temperature**(Kelvin), and `case sensitive` molecule names (e.g. TiO, Na, H2O, etc) for mixing ratios (in no particular order)

2) Must be white space delimeted 

**PICASO will auto-compute mixing ratios, determine what CIA is neceesary and compute mean molecular weight based on these headers. Take at the preloaded example below**

In [7]:
comp_file = pd.read_csv(jdi.jupiter_pt(), delim_whitespace=True) #WHITE SPACE!
#see example below
comp_file.head()

,pressure,temperature,e-,H2,H,H+,H-,VO,TiO,CO2,He,H2O,CH4,CO,NH3,N2,PH3
0,0.000001,150.87,4.500000e-38,0.837,4.500000e-38,4.500000e-38,4.500000e-38,4.500000e-38,4.500000e-38,4.500000e-38,0.163,0.000069,0.000466,4.500000e-38,0.000137,5.420000e-17,4.500000e-38
1,0.000001,149.68,4.500000e-38,0.837,4.500000e-38,4.500000e-38,4.500000e-38,4.500000e-38,4.500000e-38,4.500000e-38,0.163,0.000035,0.000466,4.500000e-38,0.000137,1.580000e-17,4.500000e-38
2,0.000002,148.40,4.500000e-38,0.837,4.500000e-38,4.500000e-38,4.500000e-38,4.500000e-38,4.500000e-38,4.500000e-38,0.163,0.000017,0.000466,4.500000e-38,0.000137,4.370000e-18,4.500000e-38
3,0.000003,147.00,4.500000e-38,0.837,4.500000e-38,4.500000e-38,4.500000e-38,4.500000e-38,4.500000e-38,4.500000e-38,0.163,0.000008,0.000466,4.500000e-38,0.000137,1.140000e-18,4.500000e-38
4,0.000004,145.46,4.500000e-38,0.837,4.500000e-38,4.500000e-38,4.500000e-38,4.500000e-38,4.500000e-38,4.500000e-38,0.163,0.000004,0.000466,4.500000e-38,0.000137,2.740000e-19,4.500000e-38


## Create 1D Spectrum

Let's create our first spectrum of Jupiter's reflected light at full phase

In [8]:
phase_angle = 0 #radians
wno, alb = jdi.picaso(all_inputs, phase_angle)
show(jpi.spectrum(wno, alb, plot_width=500))

### Option 2) Arbitrary PT and Chemistry

Sometimes for testing (or for atmospheres that we don't fully understand) an isothermal, well-mixed profile is sufficient. If we don't want to load in a full profile, we can give it a simple DataFrame with the info we need. 


In [9]:
all_inputs['atmosphere']['profile']['filepath'] = None #make sure to clear what you previously had inputed
all_inputs['atmosphere']['profile']['profile'] = pd.DataFrame({'pressure':np.logspace(-6,2,60),
                                                 'temperature':np.logspace(-6,2,60)*0+200,
                                                 "H2":np.logspace(-6,2,60)*0+0.837,
                                                 "He":np.logspace(-6,2,60)*0+0.163,
                                                 "CH4":np.logspace(-6,2,60)*0+0.000466})

In [10]:
phase_angle = 0 #phase angle in radians
wno, alb = jdi.picaso(all_inputs, phase_angle)

In [11]:
show(jpi.spectrum(wno, alb, plot_width=500))

See how the plot above is much easier to interpret than the one with the full set of molecular input. Here we can clearly see the effects of methane opacity, raman and rayleigh scattering (the next notebook will include a tutorial for more diagnostic plotting)

### Option 3: Set subset of molecules to run

Sometimes it's helpful to do a run and exclude one or several molecules from a bigger list. Take a look below

In [12]:
all_inputs['atmosphere']['profile']['profile'] = None #clear our output from above 

all_inputs['atmosphere']['profile']['filepath'] = jdi.jupiter_pt() #lets take a look again at the jupiter profile

#let's say I want to remove everything but methane and H2/He to see if there were anything other than these that were 
#influencing the spectrum.

all_inputs['atmosphere']['molecules']['whichones'] = ['He', 'H2', 'CH4']

wno, alb = jdi.picaso(all_inputs, phase_angle)
show(jpi.spectrum(wno, alb, plot_width=500))